<a href="https://colab.research.google.com/github/FIKARDAVBI/Heartbeatclassifier/blob/main/HeartBeatClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [1]:
import csv
from google.colab import drive
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataall = [pd.read_csv('/content/drive/MyDrive/filetraining/HeartbeatAI/ptbdb_' + x + '.csv') for x in ['normal','abnormal']]

In [4]:
for df in dataall:
  df.columns = list(range(len(df.columns)))

In [5]:
data = pd.concat(dataall,axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

data = data.rename({187: 'Label'},axis = 1)

In [14]:
y = data['Label'].copy()
x = data.drop('Label', axis=1).copy()

X_train, X_test, y_train,y_test = train_test_split(x, y, train_size=0.7)

In [16]:
def activationfunction(x):
  return (1.0 / (1.0 + np.exp(-x)))

def turunansigmoid(x):
  return (x * (1.0-x))

In [15]:
dat = np.array(X_train)
xdfttrain = np.fft.fft(dat)
ytrain = np.array(y_train)

In [17]:
def jst(epochs,miu,numbhl, datatrain,weightl1,weightl2 , target):
  mseplot = []
  hiddenlayer = np.empty(numbhl+1,dtype=complex)
  err = 0.0
  sse_ = 0.0
  for epoch in range(epochs):
    sse = 0.0
    for i in range(len(datatrain)):
      for k in range(numbhl):
        sum = weightl1[0][k] * 1
        for m in range(len(datatrain[0])):
          sum += weightl1[m+1][k]*datatrain[i][m]
        hiddenlayer[k+1] = activationfunction(sum)

      hiddenlayer[0] = 1.0
      output = 0.0
      for k in range(numbhl+1):
        output += weightl2[k] * hiddenlayer[k]

      output = activationfunction(output)
      err = target[i] - output
      dO = turunansigmoid(output) * err

      for k in range(numbhl+1):
        dwh = miu * hiddenlayer[k] * dO
        weightl2[k] = weightl2[k] + dwh

      dH = np.empty(numbhl,dtype=complex)

      for k in range(numbhl):
        dH[k] = turunansigmoid(hiddenlayer[k+1])*weightl2[k+1]*dO

      for k in range(numbhl):
        dwi = miu * 1 * dH[k]
        weightl1[0][k] = weightl1[0][k] + dwi
        for m in range(len(datatrain[0])):
          dwi = miu * datatrain[i][m] * dH[k]
          weightl1[m+1][k] = weightl1[m+1][k] + dwi

      sse += abs(pow(err,2))
      mse_ = sse/(i+1)
      if i%100 == 0:
        print("epoch ke : ",epoch+1,"iterasi ke : ",i,"nilai sse: ",sse, "nilai mse : ",mse_)

    sse_ += abs(pow(err,2))
    mse = sse_/(epoch+1)
    print("epoch ke : ", epoch+1, "nilai mse : ", mse)
    mseplot.append(mse)

  print("weight layer 1 ")
  print(weightl1)
  print("weight layer 2")
  print(weightl2)

  plt.plot(range(epochs), mseplot)
  plt.title('MSE vs. Epochs')
  plt.xlabel('Epochs')
  plt.ylabel('MSE')
  plt.show()

In [18]:
mseplo = []
epochs = 500
numbofhl = 124
wi = np.empty((len(xdfttrain)+1,numbofhl),dtype=complex)
wh = np.empty(numbofhl+1,dtype=complex)

for i in range(len(xdfttrain)+1):
  for k in range(numbofhl):
    wi[i][k] = random.random()

for i in range(numbofhl+1):
  wh[i] = random.random()

jst(epochs,0.3,numbofhl,xdfttrain,wi,wh,ytrain)

epoch ke :  1 iterasi ke :  0 nilai sse:  2.124101639299314e-120 nilai mse :  2.124101639299314e-120
epoch ke :  1 iterasi ke :  100 nilai sse:  34.0 nilai mse :  0.33663366336633666
epoch ke :  1 iterasi ke :  200 nilai sse:  62.0 nilai mse :  0.30845771144278605
epoch ke :  1 iterasi ke :  300 nilai sse:  97.0 nilai mse :  0.3222591362126246
epoch ke :  1 iterasi ke :  400 nilai sse:  129.0 nilai mse :  0.32169576059850374
epoch ke :  1 iterasi ke :  500 nilai sse:  159.0 nilai mse :  0.31736526946107785
epoch ke :  1 iterasi ke :  600 nilai sse:  183.0 nilai mse :  0.30449251247920134
epoch ke :  1 iterasi ke :  700 nilai sse:  214.0 nilai mse :  0.30527817403708984
epoch ke :  1 iterasi ke :  800 nilai sse:  237.0 nilai mse :  0.2958801498127341
epoch ke :  1 iterasi ke :  900 nilai sse:  264.0 nilai mse :  0.293007769145394
epoch ke :  1 iterasi ke :  1000 nilai sse:  293.0 nilai mse :  0.29270729270729273
epoch ke :  1 iterasi ke :  1100 nilai sse:  323.0 nilai mse :  0.293369663

KeyboardInterrupt: ignored